# Data Input Pipeline

## Pre-Process Images

#### 1. RGB to Contrast Stretching 

In [1]:
from PIL import Image
import numpy as np
from keras.preprocessing import image
import cv2
import glob
import shutil
import os

def normalizeRed(intensity):
    iI      = intensity
    minI    = 86
    maxI    = 230
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO

def normalizeGreen(intensity):
    iI      = intensity
    minI    = 90
    maxI    = 225
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO 

def normalizeBlue(intensity):
    iI      = intensity
    minI    = 100
    maxI    = 210
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO

path = '/home/jhanvipatel/Desktop/python/Skin/'
print(path)
for img in glob.glob("iphone_images/RGB_Contrast/*.jpg"):
    print("img:",img)
    count = 0
    imageObject     = Image.open(img)
    multiBands      = imageObject.split()
    # Apply point operations that does contrast stretching on each color band
    normalizedRedBand      = multiBands[0].point(normalizeRed)
    normalizedGreenBand    = multiBands[1].point(normalizeGreen)
    normalizedBlueBand     = multiBands[2].point(normalizeBlue)
    # Create a new image from the contrast stretched red, green and blue brands
    normalizedImage = Image.merge("RGB", (normalizedRedBand, normalizedGreenBand, normalizedBlueBand))
    im1 = normalizedImage.save(path+img)

/home/jhanvipatel/Desktop/python/Skin/
img: iphone_images/RGB_Contrast/p9.jpg
img: iphone_images/RGB_Contrast/p8.jpg
img: iphone_images/RGB_Contrast/p7.jpg
img: iphone_images/RGB_Contrast/p4.jpg
img: iphone_images/RGB_Contrast/p6.jpg
img: iphone_images/RGB_Contrast/p10.jpg
img: iphone_images/RGB_Contrast/p3.jpg
img: iphone_images/RGB_Contrast/p2.jpg
img: iphone_images/RGB_Contrast/p1.jpg
img: iphone_images/RGB_Contrast/p5.jpg


### 2. RGB Images to Contrast Stretching applied Image Equilisation with gray_scale 

In [26]:
import numpy as np
import cv2
import glob
import shutil
import os
import skimage
import pywt   
import json
import time
from PIL import Image
from skimage import io
from pathlib import Path
from keras.preprocessing import image
from matplotlib import pyplot as plt


def normalizeRed(intensity):
    iI      = intensity
    minI    = 86
    maxI    = 230
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO

def normalizeGreen(intensity):
    iI      = intensity
    minI    = 90
    maxI    = 225
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO 

def normalizeBlue(intensity):
    iI      = intensity
    minI    = 100
    maxI    = 210
    minO    = 0
    maxO    = 255
    iO      = (iI-minI)*(((maxO-minO)/(maxI-minI))+minO)
    return iO

def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_src

def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype(np.uint8)

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
        image_eq = image_eq.astype(np.uint8)
#         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.

    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'
        #         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
    return image_eq

        
path = '/home/jhanvipatel/Desktop/python/Skin/'
print(path)
for img in glob.glob("iphone_images/datapipeline/RGB_C_eqgray/*.jpg"):
    print("img:",img)
    count = 0
    imageObject     = Image.open(img)
    multiBands      = imageObject.split()
    # Apply point operations that does contrast stretching on each color band
    normalizedRedBand      = multiBands[0].point(normalizeRed)
    normalizedGreenBand    = multiBands[1].point(normalizeGreen)
    normalizedBlueBand     = multiBands[2].point(normalizeBlue)
    # Create a new image from the contrast stretched red, green and blue brands
    normalizedImage = Image.merge("RGB", (normalizedRedBand, normalizedGreenBand, normalizedBlueBand))
    im1 = normalizedImage.save(path+img)
time.sleep(5)
images=[]

for img in glob.glob("iphone_images/datapipeline/RGB_C_eqgray/*.jpg"):
        print("img_name:",img)
        imArray = cv2.imread(img)
        equalize_this(image_file=img, with_plot=False,gray_scale=True )

/home/jhanvipatel/Desktop/python/Skin/
img: iphone_images/datapipeline/RGB_C_eqgray/p9.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p8.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p7.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p4.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p6.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p10.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p3.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p2.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p1.jpg
img: iphone_images/datapipeline/RGB_C_eqgray/p5.jpg


<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p9.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p8.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p7.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original i

img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p9.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p8.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p7.jpg


<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p4.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p6.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p10.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original 

img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p4.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p6.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p10.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p3.jpg


<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p3.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p2.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
Lossy conversion from int64 to uint8. Range [0, 255]. Convert image to uint8 prior to saving to suppress this warning.
<ipython-input-26-42f5e712739f>:92: UserWarning: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p1.jpg is a low contrast image
  io.imsave(img, image_eq)             #save the equilised image by replacing the original i

img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p2.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p1.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/RGB_C_eqgray/p5.jpg


### 3. RGB Images to Heatmap applied Image Equilisation with gray_scale 

In [23]:
import cv2
from pathlib import Path
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
count = 1
def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_src

def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype(np.uint8)

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
        image_eq = image_eq.astype(np.uint8)
        #         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.

    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'
        #         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
    return image_eq

path=Path('/home/jhanvipatel/Desktop/python/Skin/')
print(path)
images=[]
for imagepath in glob.glob("iphone_images/datapipeline/RGB_h_eqgrey/*.jpg"):
    print("img:",imagepath)
    image = cv2.imread(imagepath, 0)
    heatmap2 = cv2.applyColorMap(image, cv2.COLORMAP_HOT)
    heatmapimg = cv2.imwrite(imagepath, heatmap2)
time.sleep(5)
for img in glob.glob("iphone_images/datapipeline/RGB_h_eqgrey/*.jpg"):
        print("img_name:",img)
        imArray = cv2.imread(img)
        equalize_this(image_file=img, with_plot=False,gray_scale=True )

/home/jhanvipatel/Desktop/python/Skin
img: iphone_images/datapipeline/RGB_h_eqgrey/p9.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p8.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p7.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p4.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p6.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p10.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p3.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p2.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p1.jpg
img: iphone_images/datapipeline/RGB_h_eqgrey/p5.jpg
img_name: iphone_images/datapipeline/RGB_h_eqgrey/p9.jpg
img_name: iphone_images/datapipeline/RGB_h_eqgrey/p8.jpg
img_name: iphone_images/datapipeline/RGB_h_eqgrey/p7.jpg
img_name: iphone_images/datapipeline/RGB_h_eqgrey/p4.jpg
img_name: iphone_images/datapipeline/RGB_h_eqgrey/p6.jpg
img_name: iphone_images/datapipeline/RGB_h_eqgrey/p10.jpg
img_name: iphone_images/datapipeline/RGB_h_eqgrey/p3.jpg
img_name: iphone_images/datapipeline/RGB_

### 4. RGB to Heatmap applied Image Equilisation and then image sharpening.

##### Here Image Sharpening is Optional

In [25]:
import cv2
from pathlib import Path
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from PIL import ImageFilter
count = 1
def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_src

def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype(np.uint8)

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
        image_eq = image_eq.astype(np.uint8)
        #         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.

    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'
        #         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
    return image_eq

path=Path('/home/jhanvipatel/Desktop/python/Skin/')
print(path)
images=[]
for imagepath in glob.glob("iphone_images/datapipeline/RGB_H_eq_Sh/*.jpg"):
    print("img:",imagepath)
    image = cv2.imread(imagepath, 0)
    heatmap2 = cv2.applyColorMap(image, cv2.COLORMAP_HOT)
    heatmapimg = cv2.imwrite(imagepath, heatmap2)
time.sleep(5)
for img in glob.glob("iphone_images/datapipeline/RGB_H_eq_Sh/*.jpg"):
        print("img_name:",img)
        imArray = cv2.imread(img)
        equalize_this(image_file=img, with_plot=False,gray_scale=True)
        imageName = Image.open(img);
        sharpened2 = imageName.filter(ImageFilter.SHARPEN);
        sharpened2 = sharpened2.save(img)

/home/jhanvipatel/Desktop/python/Skin
img: iphone_images/datapipeline/RGB_H_eq_Sh/p9.jpg
img: iphone_images/datapipeline/RGB_H_eq_Sh/p6.jpg
img_name: iphone_images/datapipeline/RGB_H_eq_Sh/p9.jpg
img_name: iphone_images/datapipeline/RGB_H_eq_Sh/p6.jpg


### 5. RGB Images converted to Image Sharpening and processed Equilisation on it 

In [3]:
import numpy as np
import cv2 
import skimage
import pywt   
from pathlib import Path
import glob
import json
from matplotlib import pyplot as plt
from skimage import io
from PIL import Image, ImageOps
from PIL import ImageFilter


def read_this(image_file, gray_scale=False):
    image_src = cv2.imread(image_file)
    if gray_scale:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2GRAY)
    else:
        image_src = cv2.cvtColor(image_src, cv2.COLOR_BGR2RGB)
    return image_src

def enhance_contrast(image_matrix, bins=256):
    image_flattened = image_matrix.flatten()
    image_hist = np.zeros(bins)

    # frequency count of each pixel
    for pix in image_matrix:
        image_hist[pix] += 1

    # cummulative sum
    cum_sum = np.cumsum(image_hist)
    norm = (cum_sum - cum_sum.min()) * 255
    # normalization of the pixel values
    n_ = cum_sum.max() - cum_sum.min()
    uniform_norm = norm / n_
    uniform_norm = uniform_norm.astype('int')

    # flat histogram
    image_eq = uniform_norm[image_flattened]
    # reshaping the flattened matrix to its original shape
    image_eq = np.reshape(a=image_eq, newshape=image_matrix.shape)

    return image_eq

def equalize_this(image_file, with_plot=False, gray_scale=False, bins=256):
    image_src = read_this(image_file=image_file, gray_scale=gray_scale)
    if not gray_scale:
        r_image = image_src[:, :, 0]
        g_image = image_src[:, :, 1]
        b_image = image_src[:, :, 2]

        r_image_eq = enhance_contrast(image_matrix=r_image)
        g_image_eq = enhance_contrast(image_matrix=g_image)
        b_image_eq = enhance_contrast(image_matrix=b_image)

        image_eq = np.dstack(tup=(r_image_eq, g_image_eq, b_image_eq))
        cmap_val = None
        image_eq = image_eq.astype(np.uint8)
        #         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.

    else:
        image_eq = enhance_contrast(image_matrix=image_src)
        cmap_val = 'gray'
        #         cv2.imwrite(img, image_eq)
        io.imsave(img, image_eq)             #save the equilised image by replacing the original image.
    return image_eq

images=[]

for img in glob.glob("/home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/*.jpg"):
        print("img_name:",img)
        imageName = Image.open(img);
        sharpened2 = imageName.filter(ImageFilter.SHARPEN);
        sharpened2 = sharpened2.save(img)   
        imArray = cv2.imread(img)
        equalize_this(image_file=img, with_plot=False)

img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep5.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep7.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep2.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep3.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep9.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep10.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep4.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep8.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep1.jpg
img_name: /home/jhanvipatel/Desktop/python/Skin/iphone_images/datapipeline/rgb_shrp_eq/sep6.jpg
